In [1]:
import pandas as pd
import numpy as np
import pymysql
import warnings
warnings.filterwarnings('ignore')

import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

In [2]:
sql = '''
select
om.create_time, om.order_number, om.order_method, om.order_type, cc.channel_type_id, cc.name channel_name, pa.name activity_name
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips
, tmu.id_card_num 身份证号, om.user_mobile 手机号, tmu.true_name 姓名
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_digua_business.t_channel cc on om.channel = cc.scene
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
left join db_digua_business.t_member_user tmu on tmu.id=om.user_id
where date_format(om.create_time, '%Y-%m-%d')>='2024-07-01' and date_format(om.create_time, '%Y-%m-%d')<='2025-05-31'
and om.status in (4, 8) and tprm.purchase_amount>0
'''
df = clean.query(sql)

In [5]:
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)
df.loc[:, '是否拒量'] = np.where(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True, 1, 0)
df_1105 = df[(df.下单日期>='2024-11-01')&(df.下单日期<='2025-05-31')][['order_number', '身份证号', '手机号', '姓名', '下单日期', '归属渠道', '是否拒量']]
df_0704 = df[(df.下单日期>='2024-07-01')&(df.下单日期<='2025-04-30')][['order_number', '身份证号', '手机号', '姓名', '下单日期', '归属渠道', '是否拒量']]

In [6]:
df_1105.loc[:, '身份证号_md5'] = df_1105.身份证号.apply(lambda x: clean.md5_hash(x))
df_1105.loc[:, '手机号_md5'] = df_1105.手机号.apply(lambda x: clean.md5_hash(x))
df_1105.loc[:, '姓名_md5'] = df_1105.姓名.apply(lambda x: clean.md5_hash(x))
df_1105 = df_1105[['order_number', '身份证号_md5', '手机号_md5', '姓名_md5', '下单日期', '归属渠道', '是否拒量']]
df_0704.loc[:, '身份证号_md5'] = df_0704.身份证号.apply(lambda x: clean.md5_hash(x))
df_0704.loc[:, '手机号_md5'] = df_0704.手机号.apply(lambda x: clean.md5_hash(x))
df_0704.loc[:, '姓名_md5'] = df_0704.姓名.apply(lambda x: clean.md5_hash(x))
df_0704 = df_0704[['order_number', '身份证号_md5', '手机号_md5', '姓名_md5', '下单日期', '归属渠道', '是否拒量']]

In [7]:
with pd.ExcelWriter('F:/需求/总需求/md5数据.xlsx', engine='xlsxwriter') as writer:
    df_1105.to_excel(writer, sheet_name='百融', index=False)
    df_0704.to_excel(writer, sheet_name='极信', index=False)